In [132]:
import librosa 
import matplotlib.pyplot as plt
import numpy as np
import pretty_midi
import os

# CQT Configuration 
hop_length_sec = 0.01               # 10 ms
fmin = librosa.note_to_hz('A0')     # Start at A0 (piano)
bins_per_octave = 36                # 3 bins per semitone
n_bins = 267                        # Covers A0 to C8
sr = 22050
hop_length = int(hop_length_sec * sr)

def plot_cqt_with_pianoroll(audio_path, midi_path):
    cqt_list = []
    y, _ = librosa.load(audio_path, sr=sr)

    C = librosa.cqt(
        y, sr=sr,
        hop_length=hop_length,
        fmin=fmin,
        n_bins=n_bins,
        bins_per_octave=bins_per_octave
    )
    C_dB = librosa.amplitude_to_db(np.abs(C), ref=np.max)

    # Load MIDI and get piano roll
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    fs_pianoroll = sr / hop_length  # Frame rate to match CQT
    piano_roll = midi_data.get_piano_roll(fs=fs_pianoroll)

    # Align time axes
    n_frames = min(C_dB.shape[1], piano_roll.shape[1])
    return C_dB[:, :n_frames], piano_roll[:, :n_frames]



In [133]:
# audio_path = r'note_silence\audio'
# midi_path = r'note_silence\midi'

audio_path = r'note_silence_combined'
midi_path = r'note_silence_combined'
num_sample =5
cqt_list = []
piano_list = []
# dura = '0_0312'
for k in range(1):
    for i in range(num_sample):
        if i:
            audio = os.path.join(audio_path, 'C4_durations_combined.mp3')
            midi = os.path.join(midi_path, 'C4_durations_combined.mid')
            cqt , piano = plot_cqt_with_pianoroll(audio, midi)
            cqt_list.append(cqt)
            piano_list.append(piano)
cqt_concat = np.concatenate(cqt_list, axis=1)  # along time axis
piano_concat = np.concatenate(piano_list, axis=1)

In [134]:
cqt_concat.shape
piano_concat.shape

(128, 16028)

In [135]:
# duration_sec = piano_concat.shape[1] * hop_length / sr

# # Plot setup
# fig, axs = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# # Plot CQT 
# librosa.display.specshow(
#     cqt_concat[:,:],
#     sr=sr,
#     hop_length=hop_length,
#     x_axis='s',                  
#     y_axis='cqt_note',
#     fmin=fmin,
#     bins_per_octave=bins_per_octave,
#     cmap='magma',
#     ax=axs[0]
# )
# axs[0].set_title("CQT Spectrogram")
# axs[0].label_outer()  

# # Plot Piano Roll
# pitch_min = 21
# pitch_max = 108

# axs[1].imshow(
#     piano_concat[pitch_min:pitch_max+1, :],
#     aspect='auto',
#     origin='lower',
#     cmap='gray_r',
#     interpolation='nearest',
#     extent=[0, duration_sec, pitch_min, pitch_max]
# )

# axs[1].set_xlabel('Time (seconds)')
# axs[1].set_ylabel('MIDI Pitch (Note Names)')
# axs[1].set_title('Piano Roll')
# axs[1].set_ylim(pitch_min, pitch_max)

# # Label only natural C notes
# c_notes = [n for n in range(pitch_min, pitch_max+1)
#            if pretty_midi.note_number_to_name(n).startswith('C') and '#' not in pretty_midi.note_number_to_name(n)]
# axs[1].set_yticks(c_notes)
# axs[1].set_yticklabels([pretty_midi.note_number_to_name(n) for n in c_notes])

# plt.tight_layout()
# plt.show()


In [136]:
def create_sliding_windows(cqt, window_size=9, stride=1):
    pad = window_size // 2
    cqt_padded = np.pad(cqt, ((0, 0), (pad, pad)), mode='edge')
    num_time_bins = cqt.shape[1]

    windows = np.array([
        cqt_padded[:, i:i + window_size]
        for i in range(0, num_time_bins, stride)
    ])
    
    return windows[..., np.newaxis] 

def create_sliding_windows_midi(midi, window_size=9, stride=1):
    pad = window_size // 2
    midi_padded = np.pad(midi, ((0, 0), (pad, pad)), mode='constant', constant_values=0)
    num_time_bins = midi.shape[1]

    windows = np.array([
        midi_padded[:, i:i + window_size]
        for i in range(0, num_time_bins, stride)
    ])
    
    return windows, len(windows)

In [137]:
X_input = create_sliding_windows(cqt_concat, window_size=9, stride=1)  # (num_windows, 267, 9, 1)
print("Sliding Window Output Shape:", X_input.shape)

Y_output, num_windows_midi = create_sliding_windows_midi(piano_concat, window_size=9, stride=1)
print("Sliding Window MIDI Output Shape:", Y_output.shape)

Sliding Window Output Shape: (16028, 267, 9, 1)
Sliding Window MIDI Output Shape: (16028, 128, 9)


In [138]:
(Y_output[0][60])

array([  0.,   0.,   0.,   0., 100., 100., 100., 100., 100.])

In [139]:
Y_output_flattened = np.mean(Y_output, axis=2) >= 45  # Shape: (n, 128)
# print(Y_output_flattened)
print("Xtrain : ",X_input.shape)
print("out : ",Y_output.shape)
print("Ytrain : ",Y_output_flattened.shape)

Xtrain :  (16028, 267, 9, 1)
out :  (16028, 128, 9)
Ytrain :  (16028, 128)


In [140]:
# librosa.display.specshow(cqt_concat[:, :1000], sr=sr, hop_length=hop_length,
#                             x_axis='time', y_axis='cqt_note',
#                             fmin=fmin, bins_per_octave=bins_per_octave,
#                             cmap='magma')


In [141]:
import tensorflow as tf
num_classes = 128
import datetime


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(10, (16, 2), activation='relu', padding='valid', input_shape=(267, 9, 1), name="conv1"),
    tf.keras.layers.MaxPooling2D((2, 2), name="pool1"),
    tf.keras.layers.Conv2D(20, (11, 3), activation='relu', padding='valid', name="conv2"),
    tf.keras.layers.MaxPooling2D((2, 2), name="pool2"),
    tf.keras.layers.Flatten(name="flatten"),
    tf.keras.layers.Dense(256, activation='relu', name="fc1"),
    tf.keras.layers.Dropout(0.5, name="dropout"),
    tf.keras.layers.Dense(num_classes, activation='sigmoid', name="fc2")
])

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 252, 8, 10)        330       
                                                                 
 pool1 (MaxPooling2D)        (None, 126, 4, 10)        0         
                                                                 
 conv2 (Conv2D)              (None, 116, 2, 20)        6620      
                                                                 
 pool2 (MaxPooling2D)        (None, 58, 1, 20)         0         
                                                                 
 flatten (Flatten)           (None, 1160)              0         
                                                                 
 fc1 (Dense)                 (None, 256)               297216    
                                                                 
 dropout (Dropout)           (None, 256)             

In [144]:
model.fit(X_input, Y_output_flattened, 
          epochs=100, 
          batch_size=256,
          validation_split=0.2,
          callbacks=[tensorboard_callback]
          )
model.save("C4_combine.h5")

Epoch 1/100
51/51 [==============================] - 4s 76ms/step - loss: 5.4756e-04 - accuracy: 0.5005 - val_loss: 3.9854e-04 - val_accuracy: 0.5003
Epoch 2/100
51/51 [==============================] - 4s 75ms/step - loss: 5.1383e-04 - accuracy: 0.5004 - val_loss: 8.7131e-04 - val_accuracy: 0.5003
Epoch 3/100
51/51 [==============================] - 4s 71ms/step - loss: 5.8332e-04 - accuracy: 0.5005 - val_loss: 3.3834e-04 - val_accuracy: 0.5003
Epoch 4/100
51/51 [==============================] - 4s 70ms/step - loss: 4.4064e-04 - accuracy: 0.5005 - val_loss: 3.3439e-04 - val_accuracy: 0.5003
Epoch 5/100
51/51 [==============================] - 4s 71ms/step - loss: 4.8961e-04 - accuracy: 0.5004 - val_loss: 2.8811e-04 - val_accuracy: 0.5003
Epoch 6/100
51/51 [==============================] - 4s 72ms/step - loss: 4.2451e-04 - accuracy: 0.5005 - val_loss: 2.6555e-04 - val_accuracy: 0.5003
Epoch 7/100
51/51 [==============================] - 4s 71ms/step - loss: 3.8180e-04 - accuracy: 0.5